In [47]:
import pandas as pd
import pymc as pm

df = pd.read_csv('~/data/Howell1c.csv')
# df = pd.read_csv('~/data/penguins/penguins_size.csv')
#df['adult'] = df['age'].apply(lambda x:1 if x>=20 else 0)

df = df.dropna()

target  = 'height'
predictors = ['weight','age','male']
predictors = ['age','male']

In [49]:
df4 = pd.concat([df,df,df,df])

In [51]:
df

,height,weight,age,male,adult
0,151.765,47.825606,63.0,1,1
1,139.700,36.485807,63.0,0,1
2,136.525,31.864838,65.0,0,1
3,156.845,53.041914,41.0,1,1
4,145.415,41.276872,51.0,0,1
...,...,...,...,...,...
539,145.415,31.127751,17.0,1,0
540,162.560,52.163080,31.0,1,1
541,156.210,54.062497,21.0,0,1
542,71.120,8.051258,0.0,1,0


In [52]:
df2.to_csv('~/data/Howell1c-2x.csv',index=False)

In [2]:
import numpy as np
from pandas import DataFrame

gen_age = np.random.randint(10,50,size=100)+0.1
b1 = 0.4
gen_adult = [1 if ag>=20 else 0 for ag in gen_age]
gen_adult_n = [2.1 if ag<20 else ag for ag in gen_age]

height_sd = 2
def create_height(age,adult):
    n = np.random.randn(1)[0]
    if adult:
        return 20+height_sd*n
    return age+height_sd*n

def create_heightn(age,adultn):
    return 0.1*age+10*adultn+np.random.randn(1)[0]


gen_height = [create_heightn(age,adult) for age,adult in zip(gen_age,gen_adult_n)]

df = DataFrame({'age':gen_age,'adult':gen_adult,'height':gen_height,'adultn':gen_adult_n})
df

,age,adult,height,adultn
0,10.1,0,21.738754,2.1
1,10.1,0,21.792514,2.1
2,19.1,0,21.942404,2.1
3,14.1,0,20.950959,2.1
4,11.1,0,22.878399,2.1
...,...,...,...,...
95,29.1,1,294.318558,29.1
96,20.1,1,203.009114,20.1
97,16.1,0,23.406025,2.1
98,24.1,1,243.698664,24.1


In [178]:
# Define the new row to be added
df2 = DataFrame([{'age':df['age'].mean(),'adult':np.nan,'height':df['height'].mean(),'adultn':df['adultn'].mean()},
           {'age':df['age'].mean(),'adult':df['adult'].mean(),'height':np.nan,'adultn':df['adultn'].mean()},
           {'age':df['age'].mean(),'adult':df['adult'].mean(),'height':df['height'].mean(),'adultn':np.nan}])
df2 = DataFrame([{'age':df['age'].mean(),'adult':df['adult'].mean(),'height':df['height'].mean(),'adultn':np.nan}])

# Use the loc method to add the new row to the DataFrame
df = pd.concat([df,df2])

df.tail()


,age,adult,height,adultn
96,36.10,1.00,363.265137,36.1
97,39.10,1.00,395.211918,39.1
98,44.10,1.00,444.057555,44.1
99,30.10,1.00,304.032007,30.1
0,28.99,0.74,261.257352,NaN


In [179]:
df.tail()

,age,adult,height,adultn
96,36.10,1.00,363.265137,36.1
97,39.10,1.00,395.211918,39.1
98,44.10,1.00,444.057555,44.1
99,30.10,1.00,304.032007,30.1
0,28.99,0.74,261.257352,NaN


In [3]:
cat_feature_idx,cat_feature_codes = pd.factorize(df['adult'], sort=True)

In [4]:
coords = {'adult':cat_feature_codes}


In [5]:
df.shape

(100, 4)

In [9]:
from pymc.sampling import jax

with pm.Model(coords=coords) as model:

    age_d = pm.MutableData(f'age_d',df['age'])
    df_ = pm.MutableData(f'df',df)
    adult_d = pm.MutableData('adult_d',cat_feature_idx)

with model:

    age_d = model.named_vars['age_d']
    adult_d = model.named_vars['adult_d']

    thersh = pm.Normal(f'thersh', mu=50, sigma=10)

    likelihood = pm.invlogit(age_d-thersh)
    adult = pm.Bernoulli('adult',likelihood,shape=likelihood.shape,observed=adult_d)

    a_global = pm.Normal(f'aglob', mu=0, sigma=1)
    acn = pm.Normal(f'acn', mu=a_global, sigma=1, dims='adult')

    b_global = pm.Normal(f'bglob', mu=0, sigma=1)
    bcn = pm.Normal(f'bcn', mu=b_global, sigma=1, dims='adult')

    sigma = pm.Uniform("sigma", 0, 20)
    
    mu = pm.Deterministic(f'mu_{target}', acn[adult]+bcn[adult]*age_d)
    
    target_var = pm.Normal(target, mu=mu, sigma=sigma, observed=df[target],shape=mu.shape)

    idata = pm.sample(1000, tune=1000)
    # idata = jax.sample_numpyro_nuts()
    # idata = jax.sample_blackjax_nuts()
    # pm.sample_posterior_predictive(idata, extend_inferencedata=True)


In [8]:
from pymc.sampling import jax

with pm.Model(coords=coords) as model:

    age_d = pm.MutableData(f'age_d',df['age'])
    adult_d = pm.MutableData('adult_d',cat_feature_idx)
    thersh = pm.Normal(f'thersh', mu=50, sigma=10)

    likelihood = pm.invlogit(age_d-thersh)
    adult = pm.Bernoulli('adult',likelihood,shape=likelihood.shape,observed=adult_d)

    a_global = pm.Normal(f'aglob', mu=0, sigma=1)
    acn = pm.Normal(f'acn', mu=a_global, sigma=1, dims='adult')

    b_global = pm.Normal(f'bglob', mu=0, sigma=1)
    bcn = pm.Normal(f'bcn', mu=b_global, sigma=1, dims='adult')

    sigma = pm.Uniform("sigma", 0, 20)
    
    mu = pm.Deterministic(f'mu_{target}', acn[adult]+bcn[adult]*age_d)
    
    target_var = pm.Normal(target, mu=mu, sigma=sigma, observed=df[target],shape=mu.shape)

    idata = pm.sample(1000, tune=1000)
    # idata = jax.sample_numpyro_nuts()
    # idata = jax.sample_blackjax_nuts()
    # pm.sample_posterior_predictive(idata, extend_inferencedata=True)


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 25 seconds.


In [9]:
import arviz as az
az.summary(idata)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
thersh,19.236,0.814,17.721,20.839,0.019,0.013,1857.0,1383.0,1.0
aglob,5.727,0.948,4.061,7.591,0.036,0.026,691.0,1059.0,1.0
acn[0],5.270,1.289,2.913,7.594,0.046,0.032,802.0,1133.0,1.0
acn[1],11.894,1.422,9.319,14.627,0.055,0.039,675.0,968.0,1.0
bglob,0.254,0.600,-0.891,1.354,0.015,0.013,1645.0,1220.0,1.0
...,...,...,...,...,...,...,...,...,...
mu_height[95],19.314,0.376,18.588,19.995,0.009,0.006,1682.0,1659.0,1.0
mu_height[96],17.370,0.547,16.314,18.342,0.019,0.013,852.0,1304.0,1.0
mu_height[97],20.020,0.379,19.327,20.745,0.008,0.006,2005.0,1383.0,1.0
mu_height[98],15.780,0.777,14.355,17.254,0.029,0.020,728.0,1115.0,1.0


In [106]:
fig = pm.model_to_graphviz(model)


In [191]:
df['age'] = standardize(df['age'])
df['adultn'] = standardize(df['adultn'])
df[target] = standardize(df[target])

In [16]:
from pymc.sampling import jax

my_vars = {}
with pm.Model(coords=coords) as model:

    my_vars['age_d'] = pm.MutableData(f'age_d',df['age'])
    my_vars['adult_d'] = pm.MutableData('adult_d',df['adultn'])


In [17]:

with model:

    a1 = pm.Normal(f'a1', mu=0, sigma=1)
    b1 = pm.Normal(f'b1', mu=0, sigma=1)
    mu1 = pm.Deterministic(f'mu_adult', a1+b1*my_vars[f'age_d'])

    sigma_B = pm.Exponential("sigma_B", 1)

    adult = pm.Normal('adult', mu=mu1, sigma=sigma_B, observed=my_vars[f'adult_d'])

    a2 = pm.Normal(f'a2', mu=0, sigma=1)
    b21 = pm.Normal(f'b21', mu=0, sigma=1)
    b22 = pm.Normal(f'b22', mu=0, sigma=1)
    mu2 = pm.Deterministic(f'mu_{target}', a2+b21*my_vars[f'age_d']+b22*my_vars[f'adult_d'])
    
    target_var = pm.Normal(target, mu=mu2, sigma=5, observed=df[target],shape=mu2.shape)

    idata = pm.sample(1000, tune=1000)
    # idata = jax.sample_numpyro_nuts()
    # idata = jax.sample_blackjax_nuts()
    # pm.sample_posterior_predictive(idata, extend_inferencedata=True)


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 39 seconds.


In [18]:
idata

Inference data with groups:
	> posterior
	> sample_stats
	> observed_data
	> constant_data

In [19]:
df_counterfactual = DataFrame({'age':[5.1,10.1,11,19,20,23,24,25,50,100]})
df_counterfactual['adultn'] = [20 for i in range(len(df_counterfactual))]

with model:
    pm.set_data({'age_d': df_counterfactual['age'].values})
    pm.set_data({'adult_d': df_counterfactual['adultn']})

    # use the updated values and predict outcomes and probabilities:
    # thinned_idata = idata.sel(draw=slice(None, None, 5))

    idata_2 = pm.sample_posterior_predictive(
        idata.posterior,
        var_names=[target,f'mu_{target}'],
        return_inferencedata=True,
        predictions=True,
    )

In [20]:
idata_2

Inference data with groups:
	> predictions
	> predictions_constant_data

In [46]:
az.extract(idata_2.predictions,num_samples=10).mean('sample')[target].values

array([198.22534857, 200.43218017, 198.20052808, 200.94100993,
       198.08335642, 202.0821496 , 203.0341482 , 199.01215815,
       209.07104704, 215.94831723])

In [22]:
import arviz as az

target = 'height'
target_hdi = az.hdi(idata_2.predictions)[target]
mu_mean = az.extract(idata_2.predictions).mean('sample')[f'mu_{target}']

target_hdi

<xarray.DataArray 'height' (height_dim_2: 10, hdi: 2)>
array([[188.63329949, 208.98721519],
       [190.49305685, 209.48441045],
       [190.49491092, 209.64786714],
       [191.4478771 , 210.39227457],
       [192.8705424 , 211.95281439],
       [193.2032336 , 212.31939649],
       [193.21581174, 211.66885729],
       [191.53308438, 211.0366578 ],
       [196.30484952, 217.6215218 ],
       [200.13524582, 233.34254153]])
Coordinates:
  * height_dim_2  (height_dim_2) int64 0 1 2 3 4 5 6 7 8 9
  * hdi           (hdi) <U6 'lower' 'higher'

In [168]:
df = pd.read_csv("../data/milk.csv", sep=";")
df

,clade,species,kcal.per.g,perc.fat,perc.protein,perc.lactose,mass,neocortex.perc
0,Strepsirrhine,Eulemur fulvus,0.49,16.60,15.42,67.98,1.95,55.16
1,Strepsirrhine,E macaco,0.51,19.27,16.91,63.82,2.09,NaN
2,Strepsirrhine,E mongoz,0.46,14.11,16.85,69.04,2.51,NaN
3,Strepsirrhine,E rubriventer,0.48,14.91,13.18,71.91,1.62,NaN
4,Strepsirrhine,Lemur catta,0.60,27.28,19.50,53.22,2.19,NaN
5,New World Monkey,Alouatta seniculus,0.47,21.22,23.58,55.20,5.25,64.54
6,New World Monkey,A palliata,0.56,29.66,23.46,46.88,5.37,64.54
7,New World Monkey,Cebus apella,0.89,53.41,15.80,30.79,2.51,67.64
8,New World Monkey,Saimiri boliviensis,0.91,46.08,23.34,30.58,0.71,NaN
9,New World Monkey,S sciureus,0.92,50.58,22.33,27.09,0.68,68.85


In [171]:
def standardize(series):
    """Standardize a pandas series"""
    return (series - series.mean()) / series.std()


d = pd.read_csv("../data/milk.csv", sep=";")
d["neocortex.prop"] = d["neocortex.perc"] / 100
d["logmass"] = np.log(d["mass"])

K = standardize(d["kcal.per.g"])
B = standardize(d["neocortex.prop"])
M = standardize(d["logmass"])

print(f"Number of missing values in B = {np.sum(B.isna())}")


Number of missing values in B = 12


In [173]:
# Impute missing values of B

with pm.Model() as m15_5:
    sigma_B = pm.Exponential("sigma_B", 1)
    sigma = pm.Exponential("sigma", 1)
    bM = pm.Normal("bM", 0, 0.5)
    bB = pm.Normal("bB", 0, 0.5)
    nu = pm.Normal("nu", 0, 0.5)
    a = pm.Normal("a", 0, 0.5)

    # PyMC automatically imputes missing values
    Bi = pm.Normal("Bi", nu, sigma_B, observed=B)

    mu = a + bB * Bi + bM * M

    Ki = pm.Normal("Ki", mu, sigma, observed=K)

    idata_m15_5 = pm.sample(return_inferencedata=True)
az.summary(idata_m15_5, round_to=2)


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 13 seconds.
/home/guyyos/miniconda3/envs/tusha-env/lib/python3.10/site-packages/arviz/stats/diagnostics.py:584: RuntimeWarning: invalid value encountered in scalar divide
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
/home/guyyos/miniconda3/envs/tusha-env/lib/python3.10/site-packages/arviz/stats/diagnostics.py:584: RuntimeWarning: invalid value encountered in scalar divide
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
/home/guyyos/miniconda3/envs/tusha-env/lib/python3.10/site-packages/arviz/stats/diagnostics.py:584: RuntimeWarning: invalid value encountered in scalar divide
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
/home/guyyos/miniconda3/envs/tusha-env/lib/python3.10/site-packages/arviz/stats/diagnostics.py:584: RuntimeWarning: invalid value encountered in scalar divide
  (be

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
bM,-0.54,0.20,-0.92,-0.18,0.01,0.00,1472.91,1543.26,1.0
bB,0.49,0.24,0.03,0.92,0.01,0.01,982.51,1109.84,1.0
nu,-0.04,0.21,-0.43,0.37,0.01,0.00,1706.66,1611.77,1.0
a,0.03,0.16,-0.27,0.35,0.00,0.00,2729.13,1486.38,1.0
Bi_missing[0],-0.56,0.94,-2.40,1.16,0.02,0.02,1723.73,1380.61,1.0
Bi_missing[1],-0.68,0.95,-2.44,1.14,0.02,0.02,2209.99,1212.67,1.0
Bi_missing[2],-0.71,0.93,-2.45,1.11,0.02,0.02,1827.19,1579.53,1.0
Bi_missing[3],-0.30,0.88,-1.97,1.40,0.02,0.02,2583.38,1342.62,1.0
Bi_missing[4],0.47,0.93,-1.32,2.20,0.02,0.02,2647.42,1480.35,1.0
Bi_missing[5],-0.13,0.94,-1.82,1.64,0.02,0.02,1931.76,1438.61,1.0


In [67]:
vv = df.iloc[1]['neocortex.perc']
?vv

Type:        float64
String form: nan
File:        ~/.local/lib/python3.10/site-packages/numpy/__init__.py
Docstring:  
Double-precision floating-point number type, compatible with Python `float`
and C ``double``.

:Character code: ``'d'``
:Canonical name: `numpy.double`
:Alias: `numpy.float_`
:Alias on this platform (Linux x86_64): `numpy.float64`: 64-bit precision floating-point number type: sign bit, 11 bits exponent, 52 bits mantissa.

In [77]:
type(vv), type(np.nan)

(numpy.float64, float)

In [80]:
s = pd.Series([1, 2, 3])

s.loc[0] = np.nan
s

0    NaN
1    2.0
2    3.0
dtype: float64

In [76]:
with pm.Model() as model:
    age_d = pm.MutableData(f'age_d',df['age'])
    adult_d = pm.MutableData(f'adult_d',df['adult'])

    sigma = pm.Uniform("sigma", 0, 20)
    
    b1 = pm.Normal(f'b1', mu=0, sigma=1)

    likelihood = pm.invlogit(age_d-20)

    # Bernoulli random vector with probability of success
    # given by sigmoid function and actual data as observed
    adult = pm.Bernoulli('adult',likelihood,shape=likelihood.shape,observed=adult_d)
    b21 = pm.Normal(f'b21', mu=0, sigma=5)
    b22 = pm.Normal(f'b22', mu=0, sigma=5)

    mu = pm.Deterministic(f'mu_{target}', b21*adult+b22)

    target_var = pm.Normal(target, mu=mu, sigma=sigma, shape=mu.shape, observed=df[target])

    idata = pm.sample(1000, tune=1000)
    pm.sample_posterior_predictive(idata, extend_inferencedata=True)


In [77]:
az.summary(idata)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
b1,0.045,1.032,-1.890,1.947,0.026,0.024,1579.0,1174.0,1.0
b21,5.950,0.616,4.872,7.199,0.020,0.014,956.0,1204.0,1.0
b22,13.897,0.566,12.774,14.911,0.018,0.013,953.0,1092.0,1.0
sigma,2.248,0.162,1.932,2.536,0.004,0.003,1490.0,1064.0,1.0
mu_height[0],13.897,0.566,12.774,14.911,0.018,0.013,953.0,1092.0,1.0
...,...,...,...,...,...,...,...,...,...
mu_height[95],19.847,0.239,19.392,20.272,0.005,0.004,2053.0,1610.0,1.0
mu_height[96],19.847,0.239,19.392,20.272,0.005,0.004,2053.0,1610.0,1.0
mu_height[97],19.847,0.239,19.392,20.272,0.005,0.004,2053.0,1610.0,1.0
mu_height[98],19.847,0.239,19.392,20.272,0.005,0.004,2053.0,1610.0,1.0


In [78]:
from pandas import DataFrame

df_counterfactual = DataFrame({'age':[19,20,23,24,25,50,100]})

with model:
    pm.set_data({'age_d': df_counterfactual['age'].values})
    # pm.set_data({'adult_d': df_counterfactual['adult'].values})

    # use the updated values and predict outcomes and probabilities:
    # thinned_idata = idata.sel(draw=slice(None, None, 5))

    idata_2 = pm.sample_posterior_predictive(
        idata,
        var_names=[target,f'mu_{target}'],
        return_inferencedata=True,
        predictions=True,
    )

In [79]:
import arviz as az 

az.summary(idata_2)

/home/guyyos/miniconda3/envs/tusha-env/lib/python3.10/site-packages/arviz/stats/stats.py:1351: UserWarning: Selecting first found group: predictions
  warnings.warn(f"Selecting first found group: {data.groups()[0]}")


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
height[0],15.591,3.512,9.541,22.214,0.080,0.057,1909.0,1794.0,1.0
height[1],16.778,3.815,9.660,23.002,0.087,0.062,1903.0,1874.0,1.0
height[2],19.617,2.487,14.918,24.258,0.057,0.040,1879.0,1921.0,1.0
height[3],19.874,2.342,15.609,24.213,0.051,0.036,2105.0,1927.0,1.0
height[4],19.777,2.289,15.689,24.308,0.054,0.038,1772.0,1633.0,1.0
height[5],19.884,2.223,15.403,23.764,0.049,0.035,2022.0,1924.0,1.0
height[6],19.803,2.262,15.567,23.839,0.051,0.036,1951.0,1820.0,1.0
mu_height[0],15.548,2.713,12.976,20.206,0.061,0.043,1589.0,1313.0,1.0
mu_height[1],16.864,3.011,13.045,20.195,0.069,0.049,1865.0,1444.0,1.0
mu_height[2],19.614,1.182,19.255,20.378,0.027,0.019,2125.0,1610.0,1.0


In [29]:
import arviz as az 

az.summary(idata_2)

/home/guyyos/miniconda3/envs/tusha-env/lib/python3.10/site-packages/arviz/stats/stats.py:1351: UserWarning: Selecting first found group: predictions
  warnings.warn(f"Selecting first found group: {data.groups()[0]}")


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
height[0],34.686,21.441,-5.018,74.019,0.504,0.357,1798.0,1724.0,1.0
height[1],52.473,21.600,13.757,92.267,0.486,0.344,1978.0,1968.0,1.0
height[2],68.251,21.199,29.864,108.014,0.489,0.346,1881.0,1751.0,1.0
mu_height[0],34.314,7.251,20.201,47.424,0.161,0.114,2021.0,1774.0,1.0
mu_height[1],51.774,7.365,38.186,65.664,0.163,0.116,2022.0,1859.0,1.0
mu_height[2],68.688,7.437,54.810,82.050,0.161,0.114,2144.0,1781.0,1.0
